# Practica once

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ejercicio uno

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# Cargar el conjunto de datos California Housing
housing_data = pd.read_csv('precio_casas.csv', sep=';')


# Dividir los datos en características y variable objetivo
X = housing_data.drop('Mediana precio', axis=1)
y = housing_data['Mediana precio']
X.shape, y.shape

In [ ]:
# Inicializar listas para almacenar los resultados
linear_reg_rmse = []
tree_reg_rmse = []

# Probar diferentes criterios de división de nodos para los árboles de regresión
for criterion in ['friedman_mse', 'poisson', 'squared_error', 'absolute_error']:
    # Inicializar listas para almacenar los resultados para cada criterio
    tree_reg_rmse_criterion = []

    # Dividir el conjunto de datos en entrenamiento y prueba para cada repetición
    for i in range(10):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

        # Entrenar el modelo de regresión lineal
        linear_reg = LinearRegression()
        linear_reg.fit(X_train, y_train)
        
        # Calcular el error cuadrático medio (RMSE) para el modelo de regresión lineal
        linear_reg_rmse.append(mean_squared_error(y_test, linear_reg.predict(X_test), squared=False))
        
        # Entrenar el modelo de árbol de regresión
        tree_reg = DecisionTreeRegressor(criterion=criterion)
        tree_reg.fit(X_train, y_train)
        
        # Calcular el error cuadrático medio (RMSE) para el modelo de árbol de regresión
        tree_reg_rmse_criterion.append(mean_squared_error(y_test, tree_reg.predict(X_test), squared=False))
    
    # Calcular el promedio y la desviación estándar del RMSE para el modelo de árbol de regresión con el criterio actual
    tree_reg_rmse.append((criterion, np.mean(tree_reg_rmse_criterion), np.std(tree_reg_rmse_criterion)))

In [ ]:
from collections import defaultdict
dict = defaultdict(list)
dict["model"].append("Regresión lineal")
dict["mean"].append(f"{np.mean(linear_reg_rmse):.2%}")
dict["std"].append(f"{np.std(linear_reg_rmse):.2%}")
for criterion, mean, std in tree_reg_rmse:
    dict["model"].append(f"Árbol de regresión ({criterion})")
    dict["mean"].append(f"{mean:.2%}")
    dict["std"].append(f"{std:.2%}")

df = pd.DataFrame(dict)
df

Los resultados son bastante parecidos entre sí, siendo que la regresión lineal es la que mejor resultado obtiene (73%) y el Árbol de regresión utilizando el criterio poisson el que peor (70%).

# Ejercicio dos

In [24]:
def validacion_cv(n, n_particiones=5, mezclar=True, semilla=None):
	# validación de argumentos
	if semilla is not None and not mezclar:
		raise ValueError("La semilla esta puesta pero mezclar es False, puede que tu intención era mezclarlo")
	if n <= 0:
		raise ValueError("n debe ser mayor que 0")
	if n_particiones <= 0:
		raise ValueError("n_particiones debe ser mayor que 0")
	if n_particiones > n:
		raise ValueError("n_particiones debe ser menor o igual que n")
	# se generan índices para el caso que haya que mezclar
	true_indices = np.arange(n)
	if mezclar:
		if semilla is not None:
			np.random.seed(semilla)
		np.random.shuffle(true_indices)
	partitions = []
	for k in range(n_particiones):
		test_start = k * n // n_particiones
		test_end = (k+1) * n // n_particiones
		train_indices = np.concatenate((
			np.arange(0, test_start),
			np.arange(test_end, n)
		))
		test_indices = np.arange(test_start, test_end)
		partitions.append((true_indices[train_indices], true_indices[test_indices]))
	return partitions

In [29]:
# El ejemplo dado en el enunciado:
validacion_cv(20, mezclar=False)

[(array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
  array([0, 1, 2, 3])),
 (array([ 0,  1,  2,  3,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]),
  array([4, 5, 6, 7])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7, 12, 13, 14, 15, 16, 17, 18, 19]),
  array([ 8,  9, 10, 11])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 16, 17, 18, 19]),
  array([12, 13, 14, 15])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
  array([16, 17, 18, 19]))]

In [35]:
# ejemplos con mezcla:

validacion_cv(20, mezclar=True, semilla=0)

[(array([10, 17,  6, 13,  4,  2,  5, 14,  9,  7, 16, 11,  3,  0, 15, 12]),
  array([18,  1, 19,  8])),
 (array([18,  1, 19,  8,  4,  2,  5, 14,  9,  7, 16, 11,  3,  0, 15, 12]),
  array([10, 17,  6, 13])),
 (array([18,  1, 19,  8, 10, 17,  6, 13,  9,  7, 16, 11,  3,  0, 15, 12]),
  array([ 4,  2,  5, 14])),
 (array([18,  1, 19,  8, 10, 17,  6, 13,  4,  2,  5, 14,  3,  0, 15, 12]),
  array([ 9,  7, 16, 11])),
 (array([18,  1, 19,  8, 10, 17,  6, 13,  4,  2,  5, 14,  9,  7, 16, 11]),
  array([ 3,  0, 15, 12]))]

In [36]:
# ejemplos con semilla pero sin mezcla, esto salta un aviso al programador.

validacion_cv(20, mezclar=False, semilla=0)

ValueError: La semilla esta puesta pero mezclar es False, puede que tu intención era mezclarlo